Import packages

In [1]:
import re
import requests
import pandas as pd
import numpy as np
from time import sleep
from bs4 import BeautifulSoup
import csv
import time
import sys

Set URL and directory

In [2]:
base_url = 'https://debatepolitics.com/forums/2020-us-presidential-election.210/'
p_dir = 'Downloaded_pages/2020-US-Presidential-Election/'

pages_url = []
thread_url = {}

for i in range(1,40):
    pages_url.append('page-'+str(i))

Scrape data and load onto dictonary

In [6]:
i = 1
for page in pages_url:
    page = requests.get(base_url + page)
    page_bs = BeautifulSoup(page.content, 'html.parser')
    disc = page_bs.find('div', {'class': 'structItemContainer-group js-threadList'})
    for thread in disc.find_all('div', {'class': 'structItem-title'}):
        url_val = 'https://debatepolitics.com' + thread.find('a').get('href')
        thread_url [thread.get_text()] = url_val
    
    sleep(2)
    print(f'Percentage of scrape complete {int((i/len(pages_url))*100)}%')
    i+=1

Percentage of scrape complete 2%
Percentage of scrape complete 5%
Percentage of scrape complete 7%
Percentage of scrape complete 10%
Percentage of scrape complete 12%
Percentage of scrape complete 15%
Percentage of scrape complete 17%
Percentage of scrape complete 20%
Percentage of scrape complete 23%
Percentage of scrape complete 25%
Percentage of scrape complete 28%
Percentage of scrape complete 30%
Percentage of scrape complete 33%
Percentage of scrape complete 35%
Percentage of scrape complete 38%
Percentage of scrape complete 41%
Percentage of scrape complete 43%
Percentage of scrape complete 46%
Percentage of scrape complete 48%
Percentage of scrape complete 51%
Percentage of scrape complete 53%
Percentage of scrape complete 56%
Percentage of scrape complete 58%
Percentage of scrape complete 61%
Percentage of scrape complete 64%
Percentage of scrape complete 66%
Percentage of scrape complete 69%
Percentage of scrape complete 71%
Percentage of scrape complete 74%
Percentage of scr

Scrape from each thread

In [7]:
len(thread_url)

778

In [8]:
i = 1
for title, url in thread_url.items():
    thread_html = requests.get(url)
    with open (p_dir + str(i) + '.html', 'w+') as f:
        f.write(str(thread_html.content))
    f.close()
    print(f'Percentage download completed {int((i+1)/len(thread_url))*100}%')
    i += 1
    sleep(2)

rcentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage download completed 0%
Percentage d

In [3]:
thread_data = []

for i in range(1, 779):
    with open (p_dir + str(i) + '.html', 'r') as f:
        dat = BeautifulSoup(f, 'html.parser')
        dat_com  = dat.find('div', {'class' : 'block-container lbContainer'})
        for comment in dat_com.find_all('article', {'class' : 'message message--post js-post js-inlineModContainer'}):
            comment_msg = ''.join(re.findall('[^\\\\tn]+[a-zA-Z0-9,.!@?&$%]', comment.find('div', {'class' : 'bbWrapper'}).get_text()))
            author = ''.join(re.findall('[^\\\\tn]+[a-zA-Z0-9,.!@?&$%]', comment.find('div', {'class' : 'message-userDetails'}).find('a', {'class' : 'username'}).get_text()))
            # date = comment.find('header', {'class' : 'message-attribution message-attribution--split'}).find('a').get_text('u-concealed')
            user_info = {}
            for user_extra in comment.find_all('dl', {'class' : 'pairs pairs--justified'}):
                user_info[user_extra.find('dt').get_text()] = ''.join(re.findall('[^\\\\nt]+[a-zA-Z0-9,.!@?&$%]',user_extra.find('dd').get_text()))
            # print(user_info)
            if 'Gender' in user_info and user_info['Gender'] != 'Undisclosed':
                gender = user_info['Gender']
            else:
                gender = None
            if 'Location' in user_info and user_info['Location'] != 'Undisclosed':
                location = user_info['Location']
            else:
                location = None
            if 'Political Leaning' in user_info and user_info['Political Leaning'] != 'Undisclosed':
                polLean = user_info['Political Leaning']
            else:
                polLean = None
            thread_data.append([author, comment_msg, i, gender, location, polLean])
        
        print(f'completed parsing {i}.html')
    f.close()

ml
completed parsing 36.html
completed parsing 37.html
completed parsing 38.html
completed parsing 39.html
completed parsing 40.html
completed parsing 41.html
completed parsing 42.html
completed parsing 43.html
completed parsing 44.html
completed parsing 45.html
completed parsing 46.html
completed parsing 47.html
completed parsing 48.html
completed parsing 49.html
completed parsing 50.html
completed parsing 51.html
completed parsing 52.html
completed parsing 53.html
completed parsing 54.html
completed parsing 55.html
completed parsing 56.html
completed parsing 57.html
completed parsing 58.html
completed parsing 59.html
completed parsing 60.html
completed parsing 61.html
completed parsing 62.html
completed parsing 63.html
completed parsing 64.html
completed parsing 65.html
completed parsing 66.html
completed parsing 67.html
completed parsing 68.html
completed parsing 69.html
completed parsing 70.html
completed parsing 71.html
completed parsing 72.html
completed parsing 73.html
completed

In [4]:
clm_names = ['Poster' , 'Content', 'Thread', 'Gender', 'Location', 'Political_leaning']

df = pd.DataFrame(thread_data, columns = clm_names)

df.head()

,Poster,Content,Thread,Gender,Location,Political_leaning
0,boxedin,Biden consistenly polls beter with college edu...,1,None,None,None
1,Emily L,"Boxedin, to seriously answer your question, I ...",1,None,Bedroom,None
2,The Mark,Because people with less education generally h...,1,Male,Pensylvania,Progressive
3,calamit,boxedin saidBiden consistenly polls beter with...,1,None,None,Cenrist
4,HenyBogan,boxedin saidBiden consistenly polls beter with...,1,Male,None,Cenrist


In [6]:
df.to_csv('2020_US_Presidential_Elections.csv', index = False)